# Library Imports

In [81]:
from __future__ import print_function, division
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, ReLU, Reshape,Conv2D, Flatten, Dropout, Conv2DTranspose, BatchNormalization, Embedding, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import os
import cv2
import tqdm
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
import numpy as np

In [4]:
for dirname, _, filenames in os.walk('C:/Users/Keravnos/Downloads/archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\1.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\10.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\100.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\101.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\102.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\103.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\104.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\105.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\106.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\107.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\108.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\109.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\11.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\110.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\111.png
C:/Users/Keravnos/Downloads/archive\pokemon\pokemon\112.png
C:/Users/Keravnos/Downloads/archive\pokemon\

In [84]:
def build_generator():
    
    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='int32')

    # embedding for categorical input
    li = Embedding(num_classes, latent_dim)(label)
    # linear multiplication
    scale = int(img_shape[0]/4)
    n_nodes = scale*scale
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((scale, scale, 1))(li)
    # foundation for 256x256 image but scaled down
    n_nodes = 4 * scale * scale
    gen = Dense(n_nodes)(noise)
    gen = ReLU(0.2)(gen)
    # gen = Dropout(0.5)(gen)
    gen = Reshape((scale, scale, 4))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample
    gen = Conv2DTranspose(4, (4,4), strides=(2,2), padding='same')(merge)
    gen = ReLU(0.2)(gen)
    gen = BatchNormalization(momentum=0.4, epsilon=0.002,)(gen)
    # gen = Dropout(0.5)(gen)
    # upsample
    gen = Conv2DTranspose(4, (4,4), strides=(2,2), padding='same')(gen)
    gen = ReLU(0.2)(gen)
    gen = BatchNormalization(momentum=0.4, epsilon=0.002,)(gen)
    # gen = Dropout(0.5)(gen)
    # output layer
    out_layer = Conv2D(1, (img_shape[0], img_shape[1]), activation='tanh', padding='same')(gen)
    # create Model object to return
    model = Model([noise, label], out_layer)
    model.summary()

    return model

In [91]:
def build_discriminator():

    # label input
    label = Input(shape=(1,), dtype='int32')
    # embedding for categorical input
    li = Embedding(num_classes, latent_dim)(label)
    # scale up to image dimensions with linear activation
    n_nodes = np.prod( img_shape)
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape(img_shape)(li)
    # image input
    img = Input(shape=img_shape)
    # concat label as a channel
    merge = Concatenate()([img, li])
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.2)(fe)
    # downsample
    fe = Conv2D(256, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(512, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # dropout
    fe = Dropout(0.2)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # output
    out_layer = Dense(1, activation='sigmoid')(fe)
    # define model
    model = Model([img, label], out_layer)

    sopt = Adam(learning_rate=0.0002, beta_1=0.5)

    model.compile(loss=['binary_crossentropy'],
                                optimizer=sopt,
                                metrics=['accuracy'])

    return model 


In [87]:
def prepare_data(dir):
        train_imgs = []
        train_labels = []

        folders = os.listdir(im_dir)
        for fldr in folders:
            print('COLLECTING IMAGES FROM', fldr, '...')
            next_folder_path = os.path.join(im_dir, fldr)
            img_list = os.listdir(next_folder_path)

            for img in tqdm.tqdm(img_list):
                img_path = os.path.join(next_folder_path, img)
                # load the image
                image = cv2.imread(img_path, cv2.IMREAD_COLOR)
                # resize to dimensions specified in class
                image = cv2.resize(image, (img_rows, img_cols), interpolation=cv2.INTER_AREA)
                # append image and label to export lists
                train_imgs.append(image)
                ind = folders.index(fldr)
                train_labels.append(ind)

        train_imgs = np.array(train_imgs)
        train_labels = np.array(train_labels)
        return train_imgs, train_labels

In [89]:
def train(epochs, batch_size=128, sample_interval=50):

    X_train, y_train = prepare_data(pokemon_img_path)

    # convert from ints to floats and scale from [0,255] to [-1,1]
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    # expand to 3d, e.g. add channels
    X_train = np.expand_dims(X_train, axis=3)
    y_train = y_train.reshape(-1, )

    print('X SHAPE:', X_train.shape, 'Y SHAPE:', y_train.shape)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs, labels = X_train[idx], y_train[idx]

        # Sample noise as generator input
        noise = np.random.normal(0, 1, (batch_size, 100))

        # Generate a half batch of new images
        gen_imgs = generator.predict([noise, labels])

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch([imgs, labels], valid)
        d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Condition on labels
        sampled_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)

        # Train the generator
        g_loss = combined.train_on_batch([noise, sampled_labels], valid)

        # Plot the progress
        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))

        # If at save interval => save generated image samples and model checkpoints
        if epoch % sample_interval == 0:
            sample_images(epoch)
            if epoch % 2000 == 0:
                # ---------------------
                #  SAVING CHECKPOINTS
                # ---------------------
                os.mkdir('models/model_at_{}'.format(epoch))
                model_path = os.path.join('models/model_at_{}'.format(epoch), "model_at_{}.json".format(epoch))
                # serialize model to JSON
                model_json = generator.to_json()
                with open(model_path, "w") as json_file:
                    json_file.write(model_json)
                # serialize weights to HDF5
                generator.save_weights(os.path.join('models/model_at_{}'.format(epoch), "model_at_{}.h5".format(epoch)))


In [13]:
def sample_images(epoch):
    r, c = 2, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    sampled_labels = np.arange(0, 10).reshape(-1, 1)

    gen_imgs = generator.predict([noise, sampled_labels])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
            axs[i,j].set_title("Scan: %d" % sampled_labels[cnt])
            axs[i,j].axis('off')
            cnt += 1
    
    fig.savefig(os.path.join('output', '%d.png' % epoch))
    plt.close()

In [93]:
# Input shape
img_rows = 256
img_cols = 256
channels = 3
img_shape = (img_rows, img_cols, channels)
num_classes = 10
latent_dim = 100
epochs=20000
batch_size=32
sample_interval=200


pokemon_img_path = 'C:/Users/Keravnos/Downloads/archive/pokemon/pokemon'

# create output folder
try:
    os.mkdir('output')
except FileExistsError:
    print('OUTPUT DIRECTORIES EXISTS! NO NEED TO GENERATE')
try:
    os.mkdir('models')
except FileExistsError:
    print('CHECKPOINT DIRECTORY EXISTS! NO NEED TO GENERATE')

optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.summary()

# Build the generator
generator = build_generator()

# The generator takes noise and the target label as input
# and generates the corresponding digit of that label
noise = Input(shape=(latent_dim,))
label = Input(shape=(1,))
img = generator([noise, label])

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated image as input and determines validity
# and the label of that image
valid = discriminator([img, label])

# The combined model  (stacked generator and discriminator)
# Trains generator to fool discriminator
combined = Model([noise, label], valid)
combined.compile(loss=['binary_crossentropy'],
    optimizer=optimizer)

OUTPUT DIRECTORIES EXISTS! NO NEED TO GENERATE
CHECKPOINT DIRECTORY EXISTS! NO NEED TO GENERATE
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 1, 100)       1000        input_58[0][0]                   
__________________________________________________________________________________________________
dense_23 (Dense)                (None, 1, 196608)    19857408    embedding_16[0][0]               
__________________________________________________________________________________________________
input_59 (InputLayer)           [(None, 256, 256, 3) 0                                        

ValueError: Input 0 of layer conv2d_64 is incompatible with the layer: expected axis -1 of input shape to have value 6 but received input with shape (None, 256, 256, 4)